In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,819 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [161]:
import cv2
import pytesseract
import re
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime

def extract_education(image):
    image = image[275:400, 250:310]
    image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    # image = cv2.dilate(image, None, iterations=1)
    # image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    image[image < 230] = 0
    # image = cv2.GaussianBlur(image, (, 7), 0)
    # image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 2)

    h = image.shape[0]
    part_size = h // 4
    postgraduate_part = image[0:part_size]
    university_or_college_part = image[part_size:2 * part_size]
    compulsory_or_highschool_part = image[2 * part_size:3 * part_size]
    none_completed_part = image[3 * part_size:]

    postgraduate = pytesseract.image_to_string(postgraduate_part, config="--psm 6 -c tessedit_char_whitelist=0123456789.%")
    university_or_college = pytesseract.image_to_string(university_or_college_part, config="--psm 6 -c tessedit_char_whitelist=0123456789.%")
    compulsory_or_highschool = pytesseract.image_to_string(compulsory_or_highschool_part, config="--psm 6 -c tessedit_char_whitelist=0123456789.%")
    none_completed = pytesseract.image_to_string(none_completed_part, config="--psm 6 -c tessedit_char_whitelist=0123456789.%")

    postgraduate = (postgraduate.split("\n")[0]).replace("%", "")
    university_or_college = (university_or_college.split("\n")[0]).replace("%", "")
    compulsory_or_highschool = (compulsory_or_highschool.split("\n")[0]).replace("%", "")
    none_completed = (none_completed.split("\n")[0]).replace("%", "")

    result = {
        "Postgraduate": postgraduate,
        "University or College": university_or_college,
        "Compulsory_or Highschool": compulsory_or_highschool,
        "None Completed": none_completed,
    }

    return result

def extract_age_(image):
    # TODO
    pass


def extract_employment(image):
    image = cv2.resize(image, (0, 0), fx=3, fy=3)
    image = cv2.GaussianBlur(image, (7, 7), 0)
    image[image < 230] = 0

    txt = pytesseract.image_to_string(image[:, :], config="--psm 6 -c tessedit_char_whitelist= abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.%")
    parts = txt.split('\n')
    keywords = [
        "Full-time work",
        "Unemployed",
        "Part-time work",
        "Homemaker",
        "Studies",
        "Retired",
        "Own business",
        "Leave of absence",
        "Parental leave"
    ]
    parts = [s for s in parts if any(keyword in s for keyword in keywords)]
    # print(parts)
    percentages = {}
    pattern = r'\d+\.\d+'
    for string in parts:
        for keyword in keywords:
            if keyword in string:
                found_percentages = re.findall(pattern, string)
                if found_percentages:
                    percentages[keyword] = found_percentages[0].replace("%", "")
                    break

    return percentages

def extract_interests(image):
    txt = pytesseract.image_to_string(image[:, :], config="--psm 6")
    parts = txt.split('\n')

    keywords = ["Online Services", "Mass Media", "Publishing", "Newspapers", "Retail"]
    parts = [s for s in parts if any(keyword in s for keyword in keywords)]

    percentages = {}
    regex = r'\b\d+\.\d+%'
    for string in parts:
        for keyword in keywords:
            if keyword in string:
                found_percentages = re.findall(regex, string)
                if found_percentages:
                    percentages[keyword] = found_percentages[0].replace("%", "")
                    break

    return percentages

def extract_social_media(image):
    image = cv2.resize(image, (0, 0), fx=3, fy=3)
    image = cv2.GaussianBlur(image, (7, 7), 0)
    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 7, 2)

    config = "--psm 6 -c tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.%"
    txt = pytesseract.image_to_string(image[:, :], config=config)

    keywords = ["Instagram", "YouTube", "Reddit", "Facebook", "Twitter"]
    parts = txt.split('\n')
    parts = [s for s in parts if any(keyword in s for keyword in keywords)]
    print(parts)
    percentages = {}

    regex = r'\d*\.?\d{1,2}'
    for string in parts:
        for keyword in keywords:
            if keyword in string:
                found_percentages = re.findall(regex, string)
                if found_percentages:
                    percentages[keyword] = found_percentages[0]
                    break

    return percentages

def extract_sex_(image):
    def extract_femail_percentage(image):
        txt = pytesseract.image_to_string(image, config="--psm 6")
        result = txt.split("\n")[0].replace("%", "")
        return result

    def extract_total_population(image):
        txt = pytesseract.image_to_string(image, config="--psm 6")
        result = txt.split("\n")[0].replace("M", "")
        return result

    femail_percentage = extract_femail_percentage(image[255:275, 1400:1460])
    total_population = extract_total_population(image[125:150, 1320:1420])
    return total_population, femail_percentage

def extract_age_sex(image):
    total_population, femail_percentage = extract_sex_(image)

    result = {
        "Total Population": total_population,
        "Femail": femail_percentage,
    }

    return result

def extract_household_(image):
    config = "--psm 6 -c tessedit_char_whitelist= abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.%"
    txt = pytesseract.image_to_string(image[:, :], config=config)

    parts = txt.split("\n")
    keywords = ["High", "Middle", "Low"]
    parts = [s for s in parts if any(keyword in s for keyword in keywords)]
    # print(parts)

    percentages = {}
    regex = r'\b\d+\.\d+%'
    for string in parts:
        for keyword in keywords:
            if keyword in string:
                found_percentages = re.findall(regex, string)
                if found_percentages:
                    percentages[keyword] = found_percentages[0].replace("%", "")
                    break

    return percentages

def extract_income_(image):
    # TODO
    pass

def extract_household_income(image):
    household = extract_household_(image[120:200, 1270:1440])
    return household

# Define the path to the "Images" folder, assuming it's in the current working directory
base_path = '/content/drive/MyDrive/'

# Path pattern to match all PNG files within any date-named subdirectory of "Images"
file_pattern = 'Images/*/*-amazon.com-*.png'

# Use glob to find all files that match the pattern
file_paths = glob.glob(base_path + file_pattern)

# Regex pattern to extract the category and date
pattern = r'(?P<category>[a-z_]+)-amazon\.com-(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2})\.png'

# Initialize a dictionary to hold the images, categorized by their category and date
images_by_date_category = {}

date_format = "%Y-%m-%d"

# Process each file
for k, file_path in enumerate(file_paths):
    # Convert string to datetime
    start_time = time.perf_counter()  # Start time of iteration

    # Use regex to extract category and date
    re_match = re.search(pattern, file_path)
    if re_match:
        category = re_match.group('category')
        date = f"{re_match.group('year')}-{re_match.group('month')}-{re_match.group('day')}"
        date = datetime.strptime(date, date_format)
        if date not in images_by_date_category:
            images_by_date_category[date] = {}

        # Read the image
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        print(date, category)
        # Process the image
        if category == "social_media":
            # 5 features
            result = extract_social_media(image[75:350, 40:680])
            print(len(result.keys()))
            images_by_date_category[date].update(result)

        elif category == "age_sex":
            # 2 features
            result = extract_age_sex(image)
            print(len(result.keys()))
            images_by_date_category[date].update(result)

        elif category == "education":
            # 4 features
            result = extract_education(image)
            print(len(result.keys()))
            images_by_date_category[date].update(result)

        elif category == "interests":
            # 5 features
            result = extract_interests(image[80:360, 0:750])
            print(len(result.keys()))
            images_by_date_category[date].update(result)

        elif category == "household":
            # 3 features
            result = extract_household_income(image)
            print(len(result.keys()))
            images_by_date_category[date].update(result)

        elif category == "employment":
            # 9 features
            result = extract_employment(image[70:440, 30:1060])
            print(len(result.keys()))
            images_by_date_category[date].update(result)

        else:
            continue

    else:
        print("Error")

    end_time = time.perf_counter()  # End time of iteration
    duration = end_time - start_time  # Duration of iteration
    print(f"Image {k+1}: {duration:.4f} Seconds")

2022-08-01 00:00:00 social_media
['YouTube 79.3%', 'Facebook 64.81%', 'Instagram 41.99%', 'Reddit 36.65%', 'Twitter 33.567']
5
Image 1: 2.8657 Seconds
2022-08-01 00:00:00 age_sex
2
Image 2: 0.3050 Seconds
2022-08-01 00:00:00 interests
5
Image 3: 1.3705 Seconds
2022-08-01 00:00:00 household
3
Image 4: 0.3551 Seconds
2022-08-01 00:00:00 education
4
Image 5: 0.5691 Seconds
2022-08-01 00:00:00 employment
9
Image 6: 4.8414 Seconds
2022-11-01 00:00:00 interests
5
Image 7: 1.3860 Seconds
2022-11-01 00:00:00 age_sex
2
Image 8: 0.3218 Seconds
2022-11-01 00:00:00 social_media
['YouTube 73.75%', 'Facebook 44.62%', 'Twitter 37.33%', 'Reddit 34.94%', 'Instagram 29.09%']
5
Image 9: 3.3576 Seconds
2022-11-01 00:00:00 household
3
Image 10: 0.2940 Seconds
2022-11-01 00:00:00 employment
9
Image 11: 4.9823 Seconds
2022-11-01 00:00:00 education
4
Image 12: 0.5705 Seconds
2022-10-01 00:00:00 age_sex
2
Image 13: 0.3129 Seconds
2022-10-01 00:00:00 social_media
['YouTube 75.487', 'Facebook 60%', 'Reddit 37.79

In [163]:
result = pd.DataFrame(images_by_date_category)
result

,2022-08-01,2022-11-01,2022-10-01,2022-09-01
YouTube,79.3,73.75,75.48,77.53
Facebook,64.81,44.62,60,61.87
Instagram,41.99,29.09,36.17,37.76
Reddit,36.65,34.94,37.79,39.89
Twitter,33.56,37.33,33.91,34.5
Total Population,896.5,948.3,919.5,880.2
Femail,46.36,31.33,46.36,46.36
Online Services,98.67,98.62,98.73,98.76
Mass Media,98.38,98.31,98.39,98.5
Publishing,96.36,95.12,95.95,96.29


NameError: name 'df' is not defined

In [166]:
result = pd.DataFrame(images_by_date_category).T.sort_index()
print(result.shape)
result

YouTube                     object
Facebook                    object
Instagram                   object
Reddit                      object
Twitter                     object
Total Population            object
Femail                      object
Online Services             object
Mass Media                  object
Publishing                  object
Newspapers                  object
Retail                      object
High                        object
Middle                      object
Low                         object
Postgraduate                object
University or College       object
Compulsory_or Highschool    object
None Completed              object
Full-time work              object
Unemployed                  object
Part-time work              object
Homemaker                   object
Studies                     object
Own business                object
Retired                     object
Leave of absence            object
Parental leave              object
dtype: object
(4, 28

,YouTube,Facebook,Instagram,Reddit,Twitter,Total Population,Femail,Online Services,Mass Media,Publishing,...,None Completed,Full-time work,Unemployed,Part-time work,Homemaker,Studies,Own business,Retired,Leave of absence,Parental leave
2022-08-01,79.3,64.81,41.99,36.65,33.56,896.5,46.36,98.67,98.38,96.36,...,4.89,40.52,15.12,12.58,11.81,8.01,85.3,4.53,1.43,0.69
2022-09-01,77.53,61.87,37.76,39.89,34.5,880.2,46.36,98.76,98.5,96.29,...,4.89,40.52,15.12,12.58,11.81,8.01,85.3,4.53,81.43,0.69
2022-10-01,75.48,60,36.17,37.79,33.91,919.5,46.36,98.73,98.39,95.95,...,4.89,40.52,15.12,12.58,11.81,88.01,NaN,4.53,81.43,0.69
2022-11-01,73.75,44.62,29.09,34.94,37.33,948.3,31.33,98.62,98.31,95.12,...,3.33,42.71,15.25,11.82,10.61,87.05,5.02,5.66,1.38,0.5


In [168]:
result.dtypes

YouTube                     object
Facebook                    object
Instagram                   object
Reddit                      object
Twitter                     object
Total Population            object
Femail                      object
Online Services             object
Mass Media                  object
Publishing                  object
Newspapers                  object
Retail                      object
High                        object
Middle                      object
Low                         object
Postgraduate                object
University or College       object
Compulsory_or Highschool    object
None Completed              object
Full-time work              object
Unemployed                  object
Part-time work              object
Homemaker                   object
Studies                     object
Own business                object
Retired                     object
Leave of absence            object
Parental leave              object
dtype: object

In [169]:
result.isna().sum()


YouTube                     0
Facebook                    0
Instagram                   0
Reddit                      0
Twitter                     0
Total Population            0
Femail                      0
Online Services             0
Mass Media                  0
Publishing                  0
Newspapers                  0
Retail                      0
High                        0
Middle                      0
Low                         0
Postgraduate                0
University or College       0
Compulsory_or Highschool    0
None Completed              0
Full-time work              0
Unemployed                  0
Part-time work              0
Homemaker                   0
Studies                     0
Own business                1
Retired                     0
Leave of absence            0
Parental leave              0
dtype: int64